# Homework 05 (Lakehouse with delta lake)

## Radosław Jurczak

-------------------------------------------------

A docker network `de_network` is used, created by
```{bash}
docker network create de_network
```

Minio was run with the following command:
```{bash}
docker run -p 9000:9000 -p 9090:9090 --name minio --network=de_network -v ~/minio/data:/data -e "MINIO_ROOT_USER=admin" -e "MINIO_ROOT_PASSWORD=adminadmin" quay.io/minio/minio server /data --console-address ":9090"
```

To run the code below, you'll need to create a minio bucket called `hw5`.

The notebook was run inside docker, set up by
```{bash}
docker run \
-it -d --rm \
--network=de_network \
-p 10000:8888 -p 4041:4040 \
-v "${PWD}":/home/rj/data_engineering \
quay.io/jupyter/all-spark-notebook
```

---------------------------------------------------
The data follows the example scheme outlined in the homework description: 
 - bronze layer: 3 tables per day sales of a product. The sale consists of 3 columns: `user`, `product`, and `time`. Table for product: `name` and `price`. Table for user: `name` and `location`. In total, bronze level consists of 5 tables: `sale_day_1`, `sale_day_2`, `sale_day_3`, `users`, `products`;
 - silver layer: a single table containing all sales with user and product information (union three per-day tables and join with user and product);
 - golden layer: tables with reports: the sum of money per location and amount of sales per product.

In [1]:
!pip install delta-spark
!pip install randomtimestamp

In [2]:
import datetime
import random 
from random import randint

import randomtimestamp
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StringType
from delta.tables import DeltaTable

In [3]:
spark_conf = (
    SparkConf()
    .set("spark.jars.packages", 'org.apache.hadoop:hadoop-client:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.0.0')
  
    .set("spark.driver.memory", "6g")

    
    .set("spark.hadoop.fs.s3a.endpoint", "minio:9000")
    .set("spark.hadoop.fs.s3a.access.key", "admin")
    .set("spark.hadoop.fs.s3a.secret.key", "adminadmin" )
    .set("spark.hadoop.fs.s3a.path.style.access", "true") 
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .set("spark.databricks.delta.schema.autoMerge.enabled", "true") # enable adding columns on merge
)
sc = SparkContext.getOrCreate(spark_conf)
spark = SparkSession(sc)

In [4]:
print(f"Hadoop version = {spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")
print(f"Spark version = {spark.version}")

Hadoop version = 3.3.4
Spark version = 3.5.0


In [5]:
N_SALES = (50_000, 75_000, 30_000)
N_USERS = 5000
N_PRODUCTS = 50
N_LOCATIONS = 15

#### Generate user and product tables and push them to Delta Lake

In [6]:
users = [
    (f"user_{i}",
     f"location_{random.randint(0, N_LOCATIONS)}")
    for i in range(N_USERS)
]
products = [
    (f"product_{i}",
     round(random.uniform(0.1, 1000.0), 2))
    for i in range(N_PRODUCTS)
]

user_names = [f"user_{i}" for i in range(N_USERS)]
product_names = [f"product_{i}" for i in range(N_PRODUCTS)]

user_df = spark.createDataFrame(users, ["name", "location"])
user_df.write.format("delta").mode("overwrite").save("s3a://hw5/user")
user_df.show(5)

product_df = spark.createDataFrame(products, ["name", "price"])
product_df.write.format("delta").mode("overwrite").save("s3a://hw5/product")
product_df.show(5)

+------+-----------+
|  name|   location|
+------+-----------+
|user_0| location_4|
|user_1| location_4|
|user_2| location_9|
|user_3|location_11|
|user_4| location_5|
+------+-----------+
only showing top 5 rows

+---------+------+
|     name| price|
+---------+------+
|product_0|  61.1|
|product_1| 774.5|
|product_2|  72.6|
|product_3|151.35|
|product_4|374.54|
+---------+------+
only showing top 5 rows



#### Generate sales tables for 3 days and save them to Delta Lake

In [7]:
for day, n in zip((1, 2, 3), N_SALES):
    sales = [
        (random.choice(user_names),
         random.choice(product_names),
         randomtimestamp.randomtimestamp(start=datetime.datetime.strptime(f"2023-11-1{day}", "%Y-%m-%d"), end=datetime.datetime.strptime(f"2023-11-1{day+1}", "%Y-%m-%d")))
         for _ in range(n)
    ]
    sales_df = spark.createDataFrame(sales, ["user", "product", "time"])
    sales_df.write.format("delta").mode("overwrite").save(f"s3a://hw5/sale_day_{day}")

#### Create silver layer table: all sales with user and product information 

In [8]:
sales_1_df = spark.read.format("delta").load("s3a://hw5/sale_day_1")
sales_2_df = spark.read.format("delta").load("s3a://hw5/sale_day_2")
sales_3_df = spark.read.format("delta").load("s3a://hw5/sale_day_3")

In [9]:
sales_1_df.createOrReplaceTempView("sale_day_1")
sales_2_df.createOrReplaceTempView("sale_day_2")
sales_3_df.createOrReplaceTempView("sale_day_3")

spark.sql(
    """
    (SELECT * FROM sale_day_1) UNION (SELECT * FROM sale_day_2) UNION (SELECT * FROM sale_day_3)
    """
).write.format("delta").mode("overwrite").save("s3a://hw5/all_sales")

In [10]:
all_sales_df = spark.read.format("delta").load("s3a://hw5/all_sales")
all_sales_df.count()
all_sales_df.show(5)

+---------+----------+-------------------+
|     user|   product|               time|
+---------+----------+-------------------+
|user_4752|product_17|2023-11-11 22:07:29|
|user_1734| product_9|2023-11-11 16:12:11|
| user_122|product_13|2023-11-11 08:51:41|
|user_4006|product_33|2023-11-11 07:22:05|
|user_1249|product_46|2023-11-11 05:20:48|
+---------+----------+-------------------+
only showing top 5 rows



In [11]:
all_sales_df.createOrReplaceTempView("all_sales")
user_df.createOrReplaceTempView("user")
product_df.createOrReplaceTempView("product")

spark.sql(
    """
    SELECT tmp.user, tmp.product, tmp.time, tmp.location, product.price 
    FROM 
        (SELECT all_sales.user, all_sales.product, all_sales.time, user.location 
        FROM all_sales JOIN user ON all_sales.user = user.name) tmp 
        JOIN product ON product = product.name
    """
).write.format("delta").option("mergeSchema", "true").mode("overwrite").save("s3a://hw5/all_sales")

In [12]:
all_sales_df = spark.read.format("delta").load("s3a://hw5/all_sales")
all_sales_df.count()
all_sales_df.show(5)

+--------+----------+-------------------+----------+------+
|    user|   product|               time|  location| price|
+--------+----------+-------------------+----------+------+
|user_833|product_18|2023-11-12 22:33:18|location_6|234.88|
|user_833|product_49|2023-11-12 15:58:23|location_6|250.61|
|user_833|product_10|2023-11-12 19:07:09|location_6|157.42|
|user_833|product_24|2023-11-12 00:51:51|location_6|615.58|
|user_833| product_5|2023-11-12 11:58:00|location_6|626.56|
+--------+----------+-------------------+----------+------+
only showing top 5 rows



#### Generate golden layer report tables: 
- the sum of money per location;
- the amount of sales per product.
- the number of sales made per product.

In [13]:
money_per_location_df = all_sales_df.groupby("location").agg(f.round(f.sum("price"), 2).alias("money_total")).sort("money_total", ascending=False)
money_per_location_df.show()
money_per_location_df.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("s3a://hw5/report_sales_per_location")

+-----------+-----------+
|   location|money_total|
+-----------+-----------+
| location_8| 5275035.79|
|location_13| 5011824.58|
| location_7| 4952984.58|
| location_5| 4759338.52|
| location_3| 4757949.09|
| location_9| 4748990.88|
|location_12| 4716790.45|
|location_15|  4702247.0|
|location_14| 4642540.08|
| location_4| 4638386.58|
| location_0|  4580559.6|
| location_2| 4522191.13|
| location_6| 4520359.91|
|location_10| 4383713.81|
| location_1| 4331228.59|
|location_11| 4322011.98|
+-----------+-----------+



In [14]:
sales_per_location_df = all_sales_df.groupby("product").agg(f.round(f.sum("price"), 2).alias("sales_total")).sort("sales_total", ascending=False)
sales_per_location_df.show()
sales_per_location_df.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("s3a://hw5/report_sales_per_product")

+----------+-----------+
|   product|sales_total|
+----------+-----------+
| product_6|  2908606.0|
| product_9| 2884797.84|
|product_25|  2879218.8|
|product_31|  2851241.6|
|product_33| 2785560.96|
|product_35| 2749942.08|
|product_17| 2738903.04|
|product_42| 2667326.27|
|product_22| 2635459.98|
| product_8| 2543285.15|
|product_21|  2427015.2|
| product_1|  2386234.5|
|product_27| 2334865.56|
|product_19|  2311372.3|
|product_11| 2155337.85|
|product_12| 2060566.95|
| product_5| 1922286.08|
|product_24|  1877519.0|
|product_46|  1862724.3|
|product_38|  1857887.9|
+----------+-----------+
only showing top 20 rows



In [15]:
sales_count_per_location_df = all_sales_df.groupby("product").count().withColumnRenamed("count", "sales_count").sort("sales_count", ascending=False)
sales_count_per_location_df.show()
sales_count_per_location_df.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("s3a://hw5/report_count_per_product")

+----------+-----------+
|   product|sales_count|
+----------+-----------+
|product_28|       3175|
|product_48|       3174|
|product_36|       3169|
|product_20|       3164|
|product_42|       3163|
|product_23|       3161|
|product_25|       3156|
|product_45|       3138|
|product_34|       3131|
|product_33|       3129|
|product_21|       3128|
| product_9|       3126|
|product_16|       3125|
| product_3|       3123|
|product_15|       3119|
|product_35|       3117|
|product_37|       3116|
|product_14|       3114|
|product_13|       3112|
| product_4|       3112|
+----------+-----------+
only showing top 20 rows

